In [21]:
import numpy as np
import pandas as pd
import scipy.stats as stats
from scipy.signal import welch
import os

def process_emg_folder(input_folder, output_folder, shuffle_order_file):
    """
    遍历 input_folder 下的所有 CSV 文件，处理 EMG 和 IMU 数据，并保存特征矩阵和标签到 output_folder。
    """
    # 设定采样率
    fs = 1000  # 采样率 Hz
    cycle_duration = 10  # 每个周期 10 秒
    skip_seconds = 5  # 跳过前 4 秒
    use_seconds = 5  # 需要保留的秒数

    cycle_samples = fs * cycle_duration  # 10 秒数据点数 = 10000
    skip_samples = fs * skip_seconds  # 跳过 4 秒 = 4000
    use_samples = fs * use_seconds  # 取后 6 秒 = 6000

    # 滑动窗口参数
    window_size = 200  # 200ms = 200 采样点
    step_size = 100  # 100ms = 100 采样点
    num_windows = (use_samples - window_size) // step_size + 1  # 计算窗口数

    # 创建存储文件夹
    os.makedirs(output_folder, exist_ok=True)

    # **读取 shuffle_order.xlsx**
    shuffle_df = pd.read_excel(shuffle_order_file, engine="openpyxl")
    print(shuffle_df.shape)
    
    if shuffle_df.shape[0] < 15:
        raise ValueError("标签文件数据不足 15 组！请检查 `shuffle_order.xlsx`。")

    # 存储所有数据和标签
    all_features = []
    all_labels = []

    # **遍历 CSV 文件**
    csv_files = sorted([f for f in os.listdir(input_folder) if f.endswith(".csv")])

    for file_idx, file_name in enumerate(csv_files):
        file_path = os.path.join(input_folder, file_name)
        print(f"Processing {file_name} ({file_idx+1}/{len(csv_files)})")

        # **读取数据**
        raw_data = pd.read_csv(file_path)

        # **检查通道数（忽略时间列）**
        num_channels = raw_data.shape[1] - 1
        print(f"Detected {num_channels} channels (excluding time column).")

        # **归一化（Z-score 标准化）**
        data = raw_data.iloc[:, 1:]  # 去掉时间列
        mean_vals = data.mean(axis=0)
        std_vals = data.std(axis=0)
        normalized_data = (data - mean_vals) / (std_vals + 1e-10)  # 避免除零错误

        # **更新原始数据**
        raw_data.iloc[:, 1:] = normalized_data.astype(np.float64)

        segments = []  # 存储所有分割后的数据
        labels = shuffle_df.iloc[file_idx].values.tolist()  # 获取该文件对应的 26 组标签
        num_cycles = 26  # 每个文件固定 26 段（每段 10s）

        for i in range(num_cycles):
            start_idx = i * cycle_samples + skip_samples  # 跳过前 4 秒
            end_idx = start_idx + use_samples  # 取后 6 秒

            if end_idx > len(raw_data):  # 处理不足 6000 采样点的情况
                segment = raw_data.iloc[start_idx:].values  # 取剩余数据
                pad_size = use_samples - len(segment)  # 计算填充数
                segment = np.pad(segment, ((0, pad_size), (0, 0)), mode='constant', constant_values=0)  # 填充 0
            else:
                segment = raw_data.iloc[start_idx:end_idx].values  # 正常提取数据

            # **滑动窗口**
            windows = [
                segment[j:j + window_size, 1:]  # 取 200 采样点，忽略时间列
                for j in range(0, use_samples - window_size + 1, step_size)  # 滑动步长 100
            ]
            segments.append(np.array(windows))

        # **转换为 NumPy 数组**
        segments_array = np.array(segments)  # 形状 (26, num_windows, 200, num_channels)

        # **计算特征**
        features_batches = []
        for batch_idx in range(segments_array.shape[0]):  # 26 个 batch
            batch_features = []  # 存储当前 batch 的所有窗口特征
            for window_idx in range(segments_array.shape[1]):  # 计算每个窗口
                window = segments_array[batch_idx, window_idx]  # (200, num_channels)
                features = extract_features(window)  # 计算 (15, 10)
                batch_features.append(features)  # 存入 batch

            features_batches.append(np.array(batch_features))

        features_array = np.array(features_batches)  # (26, num_windows, 15, 10)

        # **存储数据**
        all_features.append(features_array)
        all_labels.extend(labels)  # 每个文件 26个 cycle，取 shuffle_order.xlsx 里的前 26 个标签

    # **最终转换为 NumPy 数组**
    all_features = np.vstack(all_features)  # 合并所有 batch，形状 (总 batch, num_windows, 15, 10)
    all_labels = np.array(all_labels)  # (总 batch,)

    # **保存**
    np.save(os.path.join(output_folder, "feature_matrix.npy"), all_features)
    np.save(os.path.join(output_folder, "labels.npy"), all_labels)

    print(f"Feature extraction complete! Shape: {all_features.shape}")
    print(f"Labels saved: {all_labels.shape}")
    print(f"Feature matrix saved at: {output_folder}/feature_matrix.npy")
    print(f"Labels saved at: {output_folder}/labels.npy")


def extract_features(segment):
    """
    计算 EMG 和 IMU 的 15 个特征，并拼接成 (15, 10)
    """
    # **分离 EMG (前四列) 和 IMU (后六列)**
    emg_signals = segment[:, :4]  # 4 个 EMG 通道 (200, 4)
    imu_signals = segment[:, 4:]  # 6 个 IMU 通道 (200, 6)

    def compute_emg_features(signal):
        return np.array([
            np.var(signal), np.mean(np.abs(signal)), np.sqrt(np.mean(signal**2)),
            np.std(signal), np.mean(np.abs(np.diff(signal))), np.max(signal),
            np.min(signal), np.sum(np.diff(signal) > 0), np.sum(np.diff(np.sign(signal)) != 0),
            stats.kurtosis(signal), stats.skew(signal), np.sum(np.abs(np.diff(signal))),
            np.sum(signal ** 2), np.log10(np.mean(signal**2) + 1e-10),
            np.mean(welch(signal, fs=1000, nperseg=200)[1])
        ])

    def compute_imu_features(signal):
        return np.array([
            np.var(signal), np.mean(signal), np.sqrt(np.mean(signal**2)),
            np.std(signal), np.max(signal), np.min(signal),
            stats.kurtosis(signal), stats.skew(signal),
            np.mean(np.abs(np.diff(signal))), np.sum(np.abs(np.diff(signal))),
            np.sum(signal ** 2), np.log10(np.mean(signal**2) + 1e-10),
            np.mean(welch(signal, fs=1000, nperseg=200)[1]), np.median(signal), np.ptp(signal)
        ])

    emg_features = np.array([compute_emg_features(emg_signals[:, i]) for i in range(4)]).T  # (15, 4)
    imu_features = np.array([compute_imu_features(imu_signals[:, i]) for i in range(6)]).T  # (15, 6)

    return np.concatenate((emg_features, imu_features), axis=1)  # (15, 10)

# **运行函数**
root = r"data\G"
input_folder = root
output_folder = os.path.join(root, "windowed_data")
shuffle_order_file = os.path.join(root,"shuffle_order.xlsx")
process_emg_folder(input_folder, output_folder, shuffle_order_file)


(15, 26)
Processing sensor_data1.csv (1/15)
Detected 10 channels (excluding time column).


C:\Users\user\AppData\Local\Temp\ipykernel_10696\1407476850.py:61: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0       -3.775955
1       -3.775955
2       -0.214248
3       -0.214248
4       -0.214248
           ...   
45715   -0.005459
45716   -0.017740
45717   -0.017740
45718   -0.017740
45719   -0.017740
Name: EMG1, Length: 45720, dtype: float64' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  raw_data.iloc[:, 1:] = normalized_data.astype(np.float64)
C:\Users\user\AppData\Local\Temp\ipykernel_10696\1407476850.py:61: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0       -6.768217
1       -6.768217
2        0.054180
3        0.076403
4        0.054180
           ...   
45715   -0.012488
45716   -0.012488
45717   -0.034711
45718   -0.056934
45719   -0.034711
Name: EMG2, Length: 45720, dtype: float64' h

Processing sensor_data10.csv (2/15)
Detected 10 channels (excluding time column).


C:\Users\user\AppData\Local\Temp\ipykernel_10696\1407476850.py:61: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0       -5.279856
1       -5.279856
2       -0.008469
3       -0.008469
4       -0.008469
           ...   
46195   -0.043035
46196   -0.043035
46197   -0.043035
46198   -0.112168
46199   -0.146735
Name: EMG1, Length: 46200, dtype: float64' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  raw_data.iloc[:, 1:] = normalized_data.astype(np.float64)
C:\Users\user\AppData\Local\Temp\ipykernel_10696\1407476850.py:61: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0       -7.967664
1       -7.967664
2        0.052054
3        0.025846
4        0.025846
           ...   
46195   -0.052779
46196   -0.052779
46197   -0.026571
46198   -0.000362
46199   -0.000362
Name: EMG2, Length: 46200, dtype: float64' h

Processing sensor_data11.csv (3/15)
Detected 10 channels (excluding time column).


...

C:\Users\user\AppData\Local\Temp\ipykernel_10696\1407476850.py:61: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0       -5.727696
1       -5.727696
2        0.218083
3        0.218083
4        0.218083
           ...   
45838    1.287198
45839    1.249685
45840    1.193416
45841    1.118391
45842    1.005852
Name: EMG1, Length: 45843, dtype: float64' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  raw_data.iloc[:, 1:] = normalized_data.astype(np.float64)
C:\Users\user\AppData\Local\Temp\ipykernel_10696\1407476850.py:61: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0       -7.407128
1       -7.407128
2       -0.008183
3       -0.008183
4       -0.008183
           ...   
45838    0.040494
45839    0.137849
45840    0.186526
45841    0.259542
45842    0.332558
Name: EMG2, Length: 45843, dtype: float64' h

Processing sensor_data13.csv (5/15)
Detected 10 channels (excluding time column).


C:\Users\user\AppData\Local\Temp\ipykernel_10696\1407476850.py:61: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0       -5.565778
1       -5.565778
2        0.018900
3        0.018900
4        0.018900
           ...   
45864    0.000650
45865   -0.017601
45866   -0.035851
45867   -0.017601
45868   -0.054102
Name: EMG1, Length: 45869, dtype: float64' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  raw_data.iloc[:, 1:] = normalized_data.astype(np.float64)
C:\Users\user\AppData\Local\Temp\ipykernel_10696\1407476850.py:61: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0       -8.123906
1       -8.123906
2       -0.060378
3       -0.060378
4       -0.033678
           ...   
45864   -0.006978
45865   -0.033678
45866   -0.033678
45867   -0.060378
45868   -0.060378
Name: EMG2, Length: 45869, dtype: float64' h

Processing sensor_data14.csv (6/15)
Detected 10 channels (excluding time column).


C:\Users\user\AppData\Local\Temp\ipykernel_10696\1407476850.py:61: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0       -6.540702
1       -6.540702
2       -0.013084
3       -0.034556
4       -0.034556
           ...   
45710    0.029861
45711    0.051334
45712    0.051334
45713    0.051334
45714    0.029861
Name: EMG1, Length: 45715, dtype: float64' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  raw_data.iloc[:, 1:] = normalized_data.astype(np.float64)
C:\Users\user\AppData\Local\Temp\ipykernel_10696\1407476850.py:61: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0       -7.784970
1       -7.784970
2       -0.057567
3       -0.057567
4       -0.031980
           ...   
45710   -0.057567
45711   -0.057567
45712   -0.057567
45713   -0.031980
45714   -0.031980
Name: EMG2, Length: 45715, dtype: float64' h

In [15]:
import numpy as np
import pandas as pd
import scipy.stats as stats
from scipy.signal import welch
import os

def process_emg_folder(input_folder, output_folder, shuffle_order_file):
    """
    遍历 input_folder 下的所有 CSV 文件，处理 EMG 和 IMU 数据，并保存特征矩阵和标签到 output_folder。
    """
    # 设定采样率
    original_fs = 1000  # 原采样率 Hz
    target_fs = 200  # 目标采样率 Hz
    downsample_factor = original_fs // target_fs  # 降采样因子
    cycle_duration = 10  # 每个周期 10 秒
    skip_seconds = 5  # 跳过前 4 秒
    use_seconds = 5  # 需要保留的秒数

    cycle_samples = (original_fs * cycle_duration) // downsample_factor  # 2000
    skip_samples = (original_fs * skip_seconds) // downsample_factor  # 1000
    use_samples = (original_fs * use_seconds) // downsample_factor  # 1000

    # 滑动窗口参数
    window_size = 100  # 200ms = 40 采样点 (原 200, 降采样后 40)
    step_size = 50  # 100ms = 20 采样点 (原 100, 降采样后 20)
    num_windows = (use_samples - window_size) // step_size + 1  # 计算窗口数

    # 创建存储文件夹
    os.makedirs(output_folder, exist_ok=True)

    # 读取 shuffle_order.xlsx
    shuffle_df = pd.read_excel(shuffle_order_file, engine="openpyxl")
    if shuffle_df.shape[0] < 15:
        raise ValueError("标签文件数据不足 15 组！请检查 `shuffle_order.xlsx`。")

    all_features = []
    all_labels = []
    
    csv_files = sorted([f for f in os.listdir(input_folder) if f.endswith(".csv")])

    for file_idx, file_name in enumerate(csv_files):
        file_path = os.path.join(input_folder, file_name)
        print(f"Processing {file_name} ({file_idx+1}/{len(csv_files)})")

        raw_data = pd.read_csv(file_path)
        num_channels = raw_data.shape[1] - 1
        print(f"Detected {num_channels} channels (excluding time column).")

        # 降采样
        raw_data = raw_data.iloc[::downsample_factor, :].reset_index(drop=True)
        
        # 归一化
        data = raw_data.iloc[:, 1:]
        mean_vals = data.mean(axis=0)
        std_vals = data.std(axis=0)
        normalized_data = (data - mean_vals) / (std_vals + 1e-10)
        raw_data.iloc[:, 1:] = normalized_data.astype(np.float64)

        segments = []
        labels = shuffle_df.iloc[file_idx].values.tolist()
        num_cycles = 26

        for i in range(num_cycles):
            start_idx = i * cycle_samples + skip_samples
            end_idx = start_idx + use_samples

            if end_idx > len(raw_data):
                segment = raw_data.iloc[start_idx:].values
                pad_size = use_samples - len(segment)
                segment = np.pad(segment, ((0, pad_size), (0, 0)), mode='constant', constant_values=0)
            else:
                segment = raw_data.iloc[start_idx:end_idx].values

            windows = [
                segment[j:j + window_size, 1:]
                for j in range(0, use_samples - window_size + 1, step_size)
            ]
            segments.append(np.array(windows))

        segments_array = np.array(segments)
        
        features_batches = []
        for batch_idx in range(segments_array.shape[0]):
            batch_features = []
            for window_idx in range(segments_array.shape[1]):
                window = segments_array[batch_idx, window_idx]
                features = extract_features(window)
                batch_features.append(features)

            features_batches.append(np.array(batch_features))

        features_array = np.array(features_batches)

        all_features.append(features_array)
        all_labels.extend(labels)

    all_features = np.vstack(all_features)
    all_labels = np.array(all_labels)

    np.save(os.path.join(output_folder, "feature_matrix.npy"), all_features)
    np.save(os.path.join(output_folder, "labels.npy"), all_labels)

    print(f"Feature extraction complete! Shape: {all_features.shape}")
    print(f"Labels saved: {all_labels.shape}")
    print(f"Feature matrix saved at: {output_folder}/feature_matrix.npy")
    print(f"Labels saved at: {output_folder}/labels.npy")

def extract_features(segment):
    emg_signals = segment[:, :4]
    imu_signals = segment[:, 4:]

    def compute_emg_features(signal):
        return np.array([
            np.var(signal), np.mean(np.abs(signal)), np.sqrt(np.mean(signal**2)),
            np.std(signal), np.mean(np.abs(np.diff(signal))), np.max(signal),
            np.min(signal), np.sum(np.diff(signal) > 0), np.sum(np.diff(np.sign(signal)) != 0),
            stats.kurtosis(signal), stats.skew(signal), np.sum(np.abs(np.diff(signal))),
            np.sum(signal ** 2), np.log10(np.mean(signal**2) + 1e-10),
            np.mean(welch(signal, fs=200, nperseg=40)[1])
        ])

    def compute_imu_features(signal):
        return np.array([
            np.var(signal), np.mean(signal), np.sqrt(np.mean(signal**2)),
            np.std(signal), np.max(signal), np.min(signal),
            stats.kurtosis(signal), stats.skew(signal),
            np.mean(np.abs(np.diff(signal))), np.sum(np.abs(np.diff(signal))),
            np.sum(signal ** 2), np.log10(np.mean(signal**2) + 1e-10),
            np.mean(welch(signal, fs=200, nperseg=40)[1]), np.median(signal), np.ptp(signal)
        ])

    emg_features = np.array([compute_emg_features(emg_signals[:, i]) for i in range(4)]).T
    imu_features = np.array([compute_imu_features(imu_signals[:, i]) for i in range(6)]).T

    return np.concatenate((emg_features, imu_features), axis=1)

root = r"data\G"
input_folder = root
output_folder = os.path.join(root, "windowed_data")
shuffle_order_file = os.path.join(root, "shuffle_order.xlsx")
process_emg_folder(input_folder, output_folder, shuffle_order_file)

ValueError: 标签文件数据不足 15 组！请检查 `shuffle_order.xlsx`。

In [5]:
import numpy as np
import os
from sklearn.model_selection import train_test_split

def load_data(data_folder, test_size=0.2, random_state=42):
    """
    加载 `feature_matrix.npy` 和 `labels.npy` 数据，并划分训练集和测试集。

    参数：
    - data_folder: 存放数据的文件夹路径
    - test_size: 测试集比例 (默认 20%)
    - random_state: 随机种子，保证可复现性

    返回：
    - X_train: 训练集特征，形状 (train_batches, num_windows, 15, num_channels)
    - X_test: 测试集特征，形状 (test_batches, num_windows, 15, num_channels)
    - y_train: 训练集标签，形状 (train_batches,)
    - y_test: 测试集标签，形状 (test_batches,)
    """
    # **加载数据**
    feature_path = os.path.join(data_folder, "feature_matrix.npy")
    label_path = os.path.join(data_folder, "labels.npy")

    if not os.path.exists(feature_path) or not os.path.exists(label_path):
        raise FileNotFoundError("特征文件或标签文件未找到，请检查路径！")

    X = np.load(feature_path)  # 形状 (num_batches, num_windows, 15, num_channels)
    y = np.load(label_path)  # 形状 (num_batches,)

    # **数据基本信息**
    print(f"Loaded features from {feature_path}, shape: {X.shape}")
    print(f"Loaded labels from {label_path}, shape: {y.shape}")

    # **划分训练集和测试集**
    # X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state, stratify=y)
    X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=0.2, random_state=random_state
        )

    # **打印数据划分信息**
    print(f"Training set: X_train: {X_train.shape}, y_train: {y_train.shape}")
    print(f"Testing set: X_test: {X_test.shape}, y_test: {y_test.shape}")

    return X_train, X_test, y_train, y_test

# **使用示例**
data_folder = r"E:\MSC\Spring\AML\GestureLink\data\G\windowed_data"
X_train, X_test, y_train, y_test = load_data(data_folder)



Loaded features from E:\MSC\Spring\AML\GestureLink\data\G\windowed_data\feature_matrix.npy, shape: (390, 19, 15, 10)
Loaded labels from E:\MSC\Spring\AML\GestureLink\data\G\windowed_data\labels.npy, shape: (390,)
Training set: X_train: (312, 19, 15, 10), y_train: (312,)
Testing set: X_test: (78, 19, 15, 10), y_test: (78,)


----------------------------

# 处理分开的数据

In [16]:
import numpy as np
import pandas as pd
import os
import scipy.stats as stats
from scipy.signal import welch

def process_emg_folder(input_folder, output_folder, shuffle_order_file):
    """
    处理 EMG 数据文件，先按照 shuffle_order.xlsx 划分手势数据，
    然后进行窗口切分，并提取特征。
    """
    fs = 1000  # 采样率 1000Hz，每秒 1000 个数据点
    target_samples = 5000  # 每个手势的固定采样点数
    window_size = 200  # 窗口大小 400 采样点（0.4s）
    step_size = 100  # 窗口滑动步长 200 采样点（0.2s）

    # 创建输出文件夹
    os.makedirs(output_folder, exist_ok=True)

    # 读取 shuffle_order.xlsx
    shuffle_df = pd.read_excel(shuffle_order_file, engine="openpyxl")
    if shuffle_df.shape[1] < 26:
        raise ValueError("标签文件数据不足 26 组！请检查 `shuffle_order.xlsx`。")

    all_features = []
    all_labels = []
    csv_files = sorted([f for f in os.listdir(input_folder) if f.endswith(".csv")])

    for file_idx, file_name in enumerate(csv_files):
        file_path = os.path.join(input_folder, file_name)
        print(f"Processing {file_name} ({file_idx + 1}/{len(csv_files)})")

        # 读取数据
        raw_data = pd.read_csv(file_path)
        num_channels = raw_data.shape[1] - 1  # 去掉时间列
        print(f"Detected {num_channels} channels (excluding time column).")

        # 归一化数据
        data = raw_data.iloc[:, 1:]
        normalized_data = (data - data.mean(axis=0)) / (data.std(axis=0) + 1e-10)
        raw_data.iloc[:, 1:] = normalized_data.astype(np.float64)

        # 按 shuffle_order.xlsx 划分 26 组手势
        valid_segments = []
        for i in range(26):
            start_idx = shuffle_df.iloc[file_idx, i]  # 该手势起始索引
            if start_idx + target_samples <= len(raw_data):
                segment = raw_data.iloc[start_idx:start_idx + target_samples, 1:].values
            else:
                segment = raw_data.iloc[start_idx:, 1:].values
                pad_size = target_samples - len(segment)
                segment = np.pad(segment, ((0, pad_size), (0, 0)), mode='constant', constant_values=0)
            valid_segments.append(segment)

        valid_segments = np.array(valid_segments)  # 形状 (26, 5000, num_channels)

        # 进行窗口划分和特征提取
        features_batches = []
        for seg in valid_segments:
            windows = [seg[j:j + window_size, :] for j in range(0, target_samples - window_size + 1, step_size)]
            batch_features = np.array([extract_features(window) for window in windows])  # (num_windows, 15, num_channels)
            features_batches.append(batch_features)

        processed_segments = np.array(features_batches)  # (26, num_windows, 15, num_channels)

        if processed_segments.shape[0] != 26:
            print(f"Warning: {file_name} 提取到 {processed_segments.shape[0]} 个手势，不是 26 个。")

        # 存储数据
        all_features.append(processed_segments)
        all_labels.extend(shuffle_df.iloc[file_idx, :26].values.tolist())

    # 最终转换为 NumPy 数组
    all_features = np.vstack(all_features)  # (总手势数, num_windows, 15, num_channels)
    all_labels = np.array(all_labels)  # (总手势数,)

    # 保存数据
    np.save(os.path.join(output_folder, "feature_matrix.npy"), all_features)
    np.save(os.path.join(output_folder, "labels.npy"), all_labels)

    print(f"Feature extraction complete! Shape: {all_features.shape}")
    print(f"Labels saved: {all_labels.shape}")
    print(f"Feature matrix saved at: {output_folder}/feature_matrix.npy")
    print(f"Labels saved at: {output_folder}/labels.npy")

def extract_features(segment):
    """
    计算 EMG 和 IMU 的特征。
    """
    emg_signals = segment[:, :4]  # 4 个 EMG 通道
    imu_signals = segment[:, 4:]  # 6 个 IMU 通道

    def compute_emg_features(signal):
        return np.array([
            np.var(signal), np.mean(np.abs(signal)), np.sqrt(np.mean(signal**2)),
            np.std(signal), np.mean(np.abs(np.diff(signal))), np.max(signal),
            np.min(signal), np.sum(np.diff(signal) > 0), np.sum(np.diff(np.sign(signal)) != 0),
            stats.kurtosis(signal), stats.skew(signal), np.sum(np.abs(np.diff(signal))),
            np.sum(signal ** 2), np.log10(np.mean(signal**2) + 1e-10),
            np.mean(welch(signal, fs=1000, nperseg=200)[1])
        ])

    def compute_imu_features(signal):
        return np.array([
            np.var(signal), np.mean(signal), np.sqrt(np.mean(signal**2)),
            np.std(signal), np.max(signal), np.min(signal),
            stats.kurtosis(signal), stats.skew(signal),
            np.mean(np.abs(np.diff(signal))), np.sum(np.abs(np.diff(signal))),
            np.sum(signal ** 2), np.log10(np.mean(signal**2) + 1e-10),
            np.mean(welch(signal, fs=1000, nperseg=200)[1]), np.median(signal), np.ptp(signal)
        ])

    emg_features = np.array([compute_emg_features(emg_signals[:, i]) for i in range(4)]).T  # (15, 4)
    imu_features = np.array([compute_imu_features(imu_signals[:, i]) for i in range(6)]).T  # (15, 6)

    return np.concatenate((emg_features, imu_features), axis=1)  # (15, 10)

# 运行代码
root = r"data\FZH"
input_folder = root
output_folder = os.path.join(root, "processed_data")
shuffle_order_file = os.path.join(root, "shuffle_order.xlsx")
process_emg_folder(input_folder, output_folder, shuffle_order_file)

Processing sensor_data1.csv (1/10)
Detected 10 channels (excluding time column).


C:\Users\user\AppData\Local\Temp\ipykernel_10696\2422918298.py:41: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0       -0.033795
1       -0.033795
2       -0.033795
3       -0.033795
4       -0.033795
           ...   
22275   -0.033795
22276   -0.033795
22277   -0.033795
22278   -0.033795
22279   -0.033795
Name: 303, Length: 22280, dtype: float64' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  raw_data.iloc[:, 1:] = normalized_data.astype(np.float64)
C:\Users\user\AppData\Local\Temp\ipykernel_10696\2422918298.py:41: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0       -0.132256
1       -0.132256
2       -0.114027
3       -0.123142
4       -0.132256
           ...   
22275    1.079998
22276    1.079998
22277    1.061769
22278    1.061769
22279    1.070883
Name: 293, Length: 22280, dtype: float64' has

Processing sensor_data10.csv (2/10)
Detected 10 channels (excluding time column).


C:\Users\user\AppData\Local\Temp\ipykernel_10696\2422918298.py:41: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0       -1.149433
1       -0.333118
2       -1.149433
3       -1.965747
4       -0.333118
           ...   
22307    2.115824
22308    2.115824
22309    2.115824
22310    2.115824
22311    2.115824
Name: 305, Length: 22312, dtype: float64' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  raw_data.iloc[:, 1:] = normalized_data.astype(np.float64)
C:\Users\user\AppData\Local\Temp\ipykernel_10696\2422918298.py:41: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0       -0.707460
1       -0.964646
2       -1.178968
3       -1.250408
4       -1.321849
           ...   
22307    1.921554
22308    2.078723
22309    2.193028
22310    2.293045
22311    2.407350
Name: 272, Length: 22312, dtype: float64' has

Processing sensor_data2.csv (3/10)
Detected 10 channels (excluding time column).


...

Processing sensor_data8.csv (9/10)
Detected 10 channels (excluding time column).


C:\Users\user\AppData\Local\Temp\ipykernel_10696\2422918298.py:41: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0       -0.105940
1       -0.105940
2       -0.105940
3       -0.105940
4       -0.105940
           ...   
22182    0.969801
22183   -0.105940
22184   -0.105940
22185   -0.105940
22186   -0.105940
Name: 306, Length: 22187, dtype: float64' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  raw_data.iloc[:, 1:] = normalized_data.astype(np.float64)
C:\Users\user\AppData\Local\Temp\ipykernel_10696\2422918298.py:41: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0        0.085506
1        0.085506
2        0.071930
3        0.071930
4        0.099082
           ...   
22182    0.126235
22183    0.112659
22184    0.112659
22185    0.139811
22186    0.139811
Name: 309, Length: 22187, dtype: float64' has

Processing sensor_data9.csv (10/10)
Detected 10 channels (excluding time column).


C:\Users\user\AppData\Local\Temp\ipykernel_10696\2422918298.py:41: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0       -0.327510
1       -0.327510
2       -0.327510
3        0.968289
4       -0.327510
           ...   
22377   -0.327510
22378   -0.327510
22379   -0.327510
22380    0.968289
22381    0.968289
Name: 306, Length: 22382, dtype: float64' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  raw_data.iloc[:, 1:] = normalized_data.astype(np.float64)
C:\Users\user\AppData\Local\Temp\ipykernel_10696\2422918298.py:41: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0       -0.079637
1       -0.091994
2       -0.091994
3       -0.091994
4       -0.079637
           ...   
22377   -0.549196
22378   -0.598623
22379   -0.623337
22380   -0.635694
22381   -0.685121
Name: 300, Length: 22382, dtype: float64' has

Feature extraction complete! Shape: (260, 49, 15, 10)
Labels saved: (260,)
Feature matrix saved at: data\FZH\processed_data/feature_matrix.npy
Labels saved at: data\FZH\processed_data/labels.npy


In [18]:
import numpy as np
import os
from sklearn.model_selection import train_test_split

def load_data(data_folder, test_size=0.2, random_state=42):
    """
    加载 `feature_matrix.npy` 和 `labels.npy` 数据，并划分训练集和测试集。

    参数：
    - data_folder: 存放数据的文件夹路径
    - test_size: 测试集比例 (默认 20%)
    - random_state: 随机种子，保证可复现性

    返回：
    - X_train: 训练集特征，形状 (train_batches, num_windows, 15, num_channels)
    - X_test: 测试集特征，形状 (test_batches, num_windows, 15, num_channels)
    - y_train: 训练集标签，形状 (train_batches,)
    - y_test: 测试集标签，形状 (test_batches,)
    """
    # **加载数据**
    feature_path = os.path.join(data_folder, "feature_matrix.npy")
    label_path = os.path.join(data_folder, "labels.npy")

    if not os.path.exists(feature_path) or not os.path.exists(label_path):
        raise FileNotFoundError("特征文件或标签文件未找到，请检查路径！")

    X = np.load(feature_path)  # 形状 (num_batches, num_windows, 15, num_channels)
    y = np.load(label_path)  # 形状 (num_batches,)

    # **数据基本信息**
    print(f"Loaded features from {feature_path}, shape: {X.shape}")
    print(f"Loaded labels from {label_path}, shape: {y.shape}")

    # **划分训练集和测试集**
    # X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state, stratify=y)
    X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=0.2, random_state=random_state
        )

    # **打印数据划分信息**
    print(f"Training set: X_train: {X_train.shape}, y_train: {y_train.shape}")
    print(f"Testing set: X_test: {X_test.shape}, y_test: {y_test.shape}")

    return X_train, X_test, y_train, y_test

# **使用示例**
data_folder = r"E:\MSC\Spring\AML\GestureLink\data\FZH\processed_data"
X_train, X_test, y_train, y_test = load_data(data_folder)



Loaded features from E:\MSC\Spring\AML\GestureLink\data\FZH\processed_data\feature_matrix.npy, shape: (260, 49, 15, 10)
Loaded labels from E:\MSC\Spring\AML\GestureLink\data\FZH\processed_data\labels.npy, shape: (260,)
Training set: X_train: (208, 49, 15, 10), y_train: (208,)
Testing set: X_test: (52, 49, 15, 10), y_test: (52,)


In [ ]:
import numpy as np
import os
from sklearn.model_selection import train_test_split

def load_data(data_folder, test_size=0.2, random_state=42):
    """
    加载 `feature_matrix.npy` 和 `labels.npy` 数据，并划分训练集和测试集。

    参数：
    - data_folder: 存放数据的文件夹路径
    - test_size: 测试集比例 (默认 20%)
    - random_state: 随机种子，保证可复现性

    返回：
    - X_train: 训练集特征，形状 (train_batches, num_windows, 15, num_channels)
    - X_test: 测试集特征，形状 (test_batches, num_windows, 15, num_channels)
    - y_train: 训练集标签，形状 (train_batches,)
    - y_test: 测试集标签，形状 (test_batches,)
    """
    # **加载数据**
    feature_path = os.path.join(data_folder, "feature_matrix.npy")
    label_path = os.path.join(data_folder, "labels.npy")

    if not os.path.exists(feature_path) or not os.path.exists(label_path):
        raise FileNotFoundError("特征文件或标签文件未找到，请检查路径！")

    X = np.load(feature_path)  # 形状 (num_batches, num_windows, 15, num_channels)
    y = np.load(label_path)  # 形状 (num_batches,)

    # **数据基本信息**
    print(f"Loaded features from {feature_path}, shape: {X.shape}")
    print(f"Loaded labels from {label_path}, shape: {y.shape}")

    # **划分训练集和测试集**
    # X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state, stratify=y)
    X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=0.2, random_state=random_state
        )

    # **打印数据划分信息**
    print(f"Training set: X_train: {X_train.shape}, y_train: {y_train.shape}")
    print(f"Testing set: X_test: {X_test.shape}, y_test: {y_test.shape}")

    return X_train, X_test, y_train, y_test

# **使用示例**
data_folder = r"E:\MSC\Spring\AML\GestureLink\data\G\windowed_data"
X_train_g, X_test_g, y_train_g, y_test_g = load_data(data_folder)



In [ ]:
import numpy as np

X_train = np.concatenate([X_train_g, X_train], axis=0)
X_test = np.concatenate([X_test_g, X_test], axis=0)
y_train = np.concatenate([y_train_g, y_train], axis=0)
y_test = np.concatenate([y_test_g, y_test], axis=0)
